In [1]:
# Adicionando o diretório raiz na lista de diretórios para buscar módulos (p.ex: librep)
REPO_ROOT_DIR="../"
import sys,os
sys.path.append(os.path.dirname(REPO_ROOT_DIR))
from librepo.viewer_tool import ViewerTool
vt = ViewerTool()
vt.setup("http://vm.hiaac.ic.unicamp.br", "8081")
vt.auth('H-IAAC_Elds!')
exp='HIAAC4'
activity='ALL'
user="Borin"
df_acc = vt.get_csv(exp, activity, user, 'Accelerometer_Bolso Direito.csv')
df_acc = vt.get_csv(exp, activity, user, 'Accelerometer_Bolso Esquerdo.csv')
#df_acc=df_acc[0:200]
df2_acc=vt.get_csv(exp, activity, user, 'Accelerometer_Bolso Esquerdo.csv')
df_acc['sec'] = [x/1000 for x in df_acc['VideoTimelapse']]
display(df2_acc.head)

<bound method NDFrame.head of        VideoTimelapse      Timestamp   Value 1    Value 2   Value 3
0                   1  1677761144769 -0.764050  -0.433650 -9.641749
1                  27  1677761144795 -0.766444  -0.424073 -9.644144
2                  51  1677761144819 -0.797568  -0.428861 -9.653721
3                  77  1677761144845 -0.790386  -0.431256 -9.646538
4                 101  1677761144869 -0.795174  -0.438438 -9.663298
...               ...            ...       ...        ...       ...
15081          377116  1677761521884  3.732261  -9.756671 -2.340332
15082          377141  1677761521909  5.377975  -9.445425  1.257554
15083          377165  1677761521933  5.799355 -10.331579 -0.155623
15084          377191  1677761521959  4.834192 -11.313800 -0.634763
15085          377216  1677761521984  2.024297 -10.633548 -1.365293

[15086 rows x 5 columns]>

In [2]:
import os

import flask
from librepo.dashboard import DashboardApp
from librepo.video import VideoPlayer
from librepo.graph_dinamic import ScatterPlot
from librepo.graph_static import MultiTracePlot
from librepo.graph_dinamic import LinearPlot
from plotly.graph_objs import Scatter

import pandas as pd
import numpy as np
# Example usage:
if __name__ == '__main__':
    app = DashboardApp()

    ### VIDEO
    url_path = "http://127.0.0.1:8888/files/noteboks/Data/HIAAC4/Borin/ALL/20230302_094544.mp4?_xsrf=2%7Ce346787f%7Ce5bc8e78b65a33997a4e8398f47dadeb%7C1690921381"
    video_player = VideoPlayer(id="my-video", url=url_path, playing=True, loop=True, controls=True, volume=0.0, width="100%", height="100%", style={"border": "1px solid black"}, currentTime=10, secondsLoaded=30, duration=120, intervalCurrentTime=60, intervalSecondsLoaded=1000, intervalDuration=1000, seekTo=60)
    app.add_component(video_player, orientation='horizontal',width="30%", height="30%")
    video_player.controls=True
    video_player.seekTo=2
    
    ##  SCATTER PLOT 
    tsne_data = pd.DataFrame(np.random.randn(int(video_player.duration), 2), columns=['x', 'y'])
    scatter_plot = ScatterPlot(data=tsne_data, id='plot-tsne')
    app.add_component(scatter_plot, orientation='horizontal',width="50%", height="30%")

    #EVENTO PARA ATUALIZAR O GRAFICO AO MUDAR O TIMESTAMP DO VIDEO
    app.add_callback_video_scatter_update(scatter_plot,video_player) 

    


        # Criação de um objeto LinearPlot com traços iniciais
    accX = Scatter(x=df_acc['sec'], y=df_acc['Value 1'], name='trace1')
    ## CRIAR UM GRAFICO APARTIR DE UM DATAFRAME 
    linear_plot = LinearPlot(data=[accX], id='linear-plot')
    linear_plot.add_lines_from_df(df_acc,'sec',['Value 2','Value 3'])
    
    #linear_plot_up = LinearPlot(data=[accX], id='linear-plot_up')
    #linear_plot_up.update_graph(0,10)
    linear_plot.add_vLine(4)
    
    ### EVENTO QUE PERMITE ATULIZAR O VIDEO FAZENDO CLIC NA GRAFICA
    app.add_callback_video_seekTo_click_graph(video_player,linear_plot)

    # Criação de um novo traço para atualização
   # updated_trace = Scatter(x=[1, 2, 3], y=[10, 11, 12], name='trace2')

    # Atualiza o traço com ID 'trace2'
   # linear_plot.update_trace('trace2', updated_trace)


    app.add_component(linear_plot, orientation='vertical',width="100%", height="30%") 
    
   # app.add_component(linear_plot_up, orientation='horizontal',width="100%", height="30%") 
    
    
    
    

    # eventos
   # 
      
   # 
   # app.add_callback_graph_update_vLine(linear_plot_up,video_player)
    #app.add_callback_video_graph_update(linear_plot,video_player)
    
    
    


    

    app.run(host="172.17.0.2", port=8039)


/home/amparomunoz/meta_4_4_2/viewer_tool_jupyter/noteboks/../librepo/dashboard.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
